### Imports

In [1]:
import shapefile
from bs4 import BeautifulSoup
import pixie
import math
import scipy.spatial
import numpy as np
import collections

### Extracting Ireland country data
Data source: https://www.townlands.ie/page/download/

In [2]:
sf = shapefile.Reader("counties/counties.shp")
shapes = sf.shapes()

In [3]:
shape = shapes[0]
print(len(shape.parts))
print(len(shape.points))
print(shape.shapeTypeName)
print(shape.points[0])
print(max([len(shape.parts) for shape in shapes]))

1
8938
POLYGON
(-7.3395627, 54.1467174)
283


In [4]:
polys = []
for shape in shapes:
    parts = shape.parts[:]
    parts.append(len(shape.points))
    for i in range(len(parts)-1):
        polys.append(shape.points[parts[i]:parts[i+1]])

### Extracting triangulation pillar data
Downloaded the data here: http://www.trigpointing-ireland.org.uk/index.php

In [5]:
with open("TPI.gpx.xml", "r") as f:
    xml = BeautifulSoup(f.read())

In [ ]:
print(xml.prettify())

In [7]:
pillarElements = xml.findAll("wpt")
pillars = []
for pillarEle in pillarElements:
    lat, lon = map(lambda name: float(pillarEle.attrs[name]),
                   ["lat", "lon"])
    descEle = pillarEle.find("desc")
    name = descEle.contents[0].strip()
    pillars.append(dict(lat=lat, lon=lon, name=name))

In [ ]:
pillars

In [9]:
print(len(pillars))

868


### Defining the edges of the map & normalising the coordinates
These calculations should only be run once, otherwise it'll mess up the coordinates and they'll have to be reloaded.

In [10]:
min_lon, max_lon = math.inf, -math.inf
min_lat, max_lat = math.inf, -math.inf

for poly in polys:
    for point in poly:
        lon, lat = point
        min_lon = min(lon, min_lon)
        max_lon = max(lon, max_lon)
        min_lat = min(lat, min_lat)
        max_lat = max(lat, max_lat)

In [11]:
print(min_lon, max_lon, min_lat, max_lat)
print()
print()

-10.6626168 -5.4268157 51.388867 55.4352974




In [12]:
gap = 0.3
min_lon -= gap
max_lon += gap
min_lat -= gap
max_lat += gap

Normalise the boundaries of the country.

In [13]:
for poly in polys:
    for i in range(len(poly)):
        lon, lat = poly[i]
        lon = (lon-min_lon)/(max_lon-min_lon)
        lat = (lat-min_lat)/(max_lat-min_lat)
        poly[i] = (lon, lat)

And normalise the coordinates of the pillars.

In [14]:
for pillar in pillars:
    lon, lat = pillar["lon"], pillar["lat"]
    # Nice copy & paste job.
    lon = (lon-min_lon)/(max_lon-min_lon)
    lat = (lat-min_lat)/(max_lat-min_lat)
    pillar["lon"], pillar["lat"] = lon, lat

### Draw the map

In [15]:
def to_image_coords(W, H, x, y):
    return x*W, (1-y)*H

def draw_line(ctx, W, H, P1, P2):
    # P1 and P2 are assumed to have normalised x & y coordinates
    # in the range [0,1], these are converted to points in the image.
    # Also has to invert the y value since positive y direction is down
    # in drawing interfaces (usually).
    ctx.stroke_segment(*to_image_coords(W, H, *P1),
                       *to_image_coords(W, H, *P2))

def draw_pillar(image, paint, W, H, P):
    path = pixie.Path()
    path.polygon(*to_image_coords(W, H, *P), 3, 3)
    image.fill_path(path, paint)

def draw_ireland(ctx, W, H, polys):
    paint = pixie.Paint(pixie.SOLID_PAINT)
    paint.color = pixie.parse_color("#000000")

    ctx.stroke_style = paint
    ctx.line_width = 1

    for poly in polys:
        for i in range(len(poly)-1):
            draw_line(ctx, W, H, poly[i], poly[i+1])
        draw_line(ctx, W, H, poly[-1], poly[0])

def draw_pillars(ctx, image, W, H, pillars):
    pillar_paint = pixie.Paint(pixie.SOLID_PAINT)
    pillar_paint.color = pixie.Color(0, 0, 1, 1)
    ctx.fill_style = pillar_paint
    for pillar in pillars:
        draw_pillar(image, pillar_paint,
                    W, H, (pillar["lon"], pillar["lat"]))

W, H = 300, 360
image = pixie.Image(W, H)
image.fill(pixie.Color(1, 1, 1, 1))
ctx = image.new_context()

draw_ireland(ctx, W, H, polys)
draw_pillars(ctx, image, W, H, pillars)
for pillar in pillars:
    if pillar["name"] == "Benbulbin":
        pillar_paint = pixie.Paint(pixie.SOLID_PAINT)
        pillar_paint.color = pixie.Color(1, 0, 0, 1)
        ctx.fill_style = pillar_paint
        draw_pillar(image, pillar_paint, W, H, (pillar["lon"], pillar["lat"]))
        break

image.write_file("/tmp/map.png")

### Create Delaunay triangulation

In [16]:
tri_data = np.array(
    [(pillar["lon"], pillar["lat"]) for pillar in pillars])
tri = scipy.spatial.Delaunay(tri_data)

In [17]:
tri.simplices[:5]

array([[796, 799, 821],
       [799, 823, 821],
       [826, 798, 796],
       [864, 841, 119],
       [148, 138, 146]], dtype=int32)

In [18]:
def euclidean_dist(pillar1, pillar2):
    return math.sqrt(
        sum((pillar2[n]-pillar1[n])**2 for n in ["lon", "lat"]))

length_threshold = 0.08
triangles = []
for triangle in tri.simplices:
    bad_triangle = False
    for (i, j) in [(0, 1), (1, 2), (2, 0)]:
        P1 = pillars[triangle[i]]
        P2 = pillars[triangle[j]]
        if euclidean_dist(P1, P2) >= length_threshold:
            bad_triangle = True
    if not bad_triangle:
        # Filtering out triangles that have a side length that
        # is unrealistically large.
        triangles.append(triangle)

# Now confirm that we haven't left out any pillars.
covered_pillars = set()
for triangle in triangles:
    for i in triangle:
        covered_pillars.add(i)
missing_pillars = set(range(len(pillars))) - covered_pillars
print(f"Missing {len(missing_pillars)} pillars:")
for i in missing_pillars:
    print(" ", pillars[i]["name"])

Missing 2 pillars:
  Wolftrap Mountain
  Bannanimma (Carrickaveilty)


### Draw pillar mesh!

In [19]:
def draw_mesh(ctx, W, H, pillars, triangles):
    paint = pixie.Paint(pixie.SOLID_PAINT)
    paint.color = pixie.Color(1, 0, 0, 0.7)

    ctx.stroke_style = paint
    ctx.line_width = 0.5
    for triangle in triangles:
        for (i, j) in [(0, 1), (1, 2), (2, 0)]:
            P1 = pillars[triangle[i]]
            P2 = pillars[triangle[j]]
            draw_line(ctx, W, H,
                      (P1["lon"], P1["lat"]),
                      (P2["lon"], P2["lat"]))

W, H = 300, 360
image = pixie.Image(W, H)
image.fill(pixie.Color(1, 1, 1, 1))
ctx = image.new_context()

#draw_ireland(ctx, W, H, polys)
draw_pillars(ctx, image, W, H, pillars)
draw_mesh(ctx, W, H, pillars, triangles)

image.write_file("/tmp/mesh-only-map.png")

### Use law of sines  to construct mesh

First, pick 2 starting pillars. They must be part of the same triangle.

In [20]:
for i, pillar in enumerate(pillars):
    if pillar["name"] == "Scalp Mountain":
        a_index = i
        break
for i, pillar in enumerate(pillars):
    if pillar["name"] == "Creehennan Hill":
        b_index = i
        break
print(a_index, b_index)
for triangle in triangles:
    if a_index in triangle and b_index in triangle:
        first_triangle = triangle
        print("Yay, they're in this triangle:", triangle)
        break

763 766
Yay, they're in this triangle: [766 763  87]


Calculate the distance between them, and initialise the data structures we need to propagate the distance through the triangle mesh. The amount of error in that distance measurement can be adjusted.

In [21]:
# This is used to track the distances between pillars, i.e. the edge lengths
# in the triangle mesh.
distance_map = {}
def get_distance_key(p1_id, p2_id):
    return tuple(sorted([p1_id, p2_id]))
def add_dist(distance_map, p1_id, p2_id, dist):
    distance_map[get_distance_key(p1_id, p2_id)] = dist
def get_dist(distance_map, p1_id, p2_id):
    return distance_map[get_distance_key(p1_id, p2_id)]
def dist_calculated_yet(distance_map, p1_id, p2_id):
    return get_distance_key(p1_id, p2_id) in distance_map
def get_pillar_coords(pillars, index):
    return np.array([pillars[index]["lon"], pillars[index]["lat"]])
def calc_final_coords(a_index, b_index, pillars, final_coords, dist_map):
    """Calculates coordinates of a pillar B based on the final coordinates
    of a pillar B. Assumes that the distance between them has been calculated already,
    and of course that the final position of A is already fixed."""
    a_position = final_coords[a_index]
    direction_ab = get_pillar_coords(pillars, b_index) - a_position
    direction_ab /= np.linalg.norm(direction_ab)
    return a_position + get_dist(dist_map, a_index, b_index)*direction_ab
error = 0.25
# Making the distance smaller so it doesn't become massive.
base_distance = (1-error)*euclidean_dist(pillars[a_index], pillars[b_index])
add_dist(distance_map, a_index, b_index, base_distance)

visited_pillars = set([a_index, b_index])
triangles_by_pillar = collections.defaultdict(list)
for triangle in triangles:
    for pillar in triangle:
        triangles_by_pillar[pillar].append(triangle)

# For the pillars we add, track which triangle they're part of.
connecting_triangles = {a_index: first_triangle, b_index: first_triangle}

# Also, track the final coordinates of all the pillars
# we visit when reconstructing the mesh. The first 2 pillars
# are already fixed.
final_coords = {}
final_coords[a_index] = get_pillar_coords(pillars, a_index)
final_coords[b_index] = calc_final_coords(
    a_index, b_index, pillars, final_coords, distance_map)

def at_least_one_sidelength_estimated(triangle, dist_map):
    for (i, j) in [(0, 1), (1, 2), (2, 0)]:
        if dist_calculated_yet(dist_map, triangle[i], triangle[j]):
            return True
    return False
def fetch_new_candidates(triangles_by_pillar, dist_map, pillar, connecting_triangles):
    neighbours = set()
    for triangle in triangles_by_pillar[pillar]:
        for neighbour in triangle:
            if (neighbour not in visited_pillars
                 and at_least_one_sidelength_estimated(triangle, dist_map)):
                neighbours.add(neighbour)
                connecting_triangles[neighbour] = triangle
    return neighbours
candidate_pillars = set()
for pillar in visited_pillars:
    candidate_pillars |= fetch_new_candidates(
        triangles_by_pillar, distance_map, pillar, connecting_triangles)

In [22]:
print(candidate_pillars)
print(connecting_triangles)

{764, 87}
{763: array([766, 763,  87], dtype=int32), 766: array([766, 763,  87], dtype=int32), 87: array([766, 763,  87], dtype=int32), 764: array([763, 766, 764], dtype=int32)}


Now propagate the distance through the triangle mesh. We can only propagate to a pillar that is part of a triangle with 2 already-connected pillars (since that means we have a side length of the triangle and can determine the remaining distances).

In [23]:
def update_distances(p_id, left_id, right_id, pillars, distance_map):
    plr_angle = math.radians(calc_angle(p_id, left_id, right_id, pillars))
    lrp_angle = math.radians(calc_angle(left_id, right_id, p_id, pillars))
    rpl_angle = math.radians(180 - math.degrees(plr_angle) - math.degrees(lrp_angle))
    #print("UPDATE_DISTANCES") # DEBUG
    #print(pillars[p_id]["name"], pillars[left_id]["name"], pillars[right_id]["name"])
    #print(math.degrees(plr_angle))
    #print(math.degrees(lrp_angle))
    #print(math.degrees(rpl_angle))
    d = get_dist(distance_map, left_id, right_id)
    # Law of sines.
    pl_dist = d*math.sin(lrp_angle)/math.sin(rpl_angle)
    pr_dist = d*math.sin(plr_angle)/math.sin(rpl_angle)
    add_dist(distance_map, p_id, left_id, pl_dist)
    add_dist(distance_map, p_id, right_id, pr_dist)

def calc_angle(p_1, p_2, p_3, pillars):
    """Calculates the angle formed by p_1->p_2->p_3. They should be in clockwise order, I think."""
    pill1, pill2, pill3 = pillars[p_1], pillars[p_2], pillars[p_3]
    line_12 = line_between(pill1, pill2)
    line_23 = line_between(pill2, pill3)
    # I was lazy, modified angle code from here...
    # https://stackoverflow.com/questions/35176451/python-code-to-calculate-angle-between-three-point-using-their-3d-coordinates
    dot = np.dot(line_12, line_23)
    angle = math.degrees(np.arccos(abs(dot) / (np.linalg.norm(line_12) * np.linalg.norm(line_23))))
    if dot > 0:
        # This is supposed to ensure that we get
        # the internal angle of the triangle. Hard to explain
        # without drawing everything out.
        angle = 180 - angle
    return angle

def line_between(pill1, pill2):
    return np.array([pill2["lon"] - pill1["lon"],
                     pill2["lat"] - pill1["lat"]])

def clockwise_order(p_index, left_index, right_index, pillars):
    p1 = pillars[p_index]
    p2 = pillars[left_index]
    p3 = pillars[right_index]
    return np.linalg.det(np.array([
        [p1["lon"], p1["lat"], 1],
        [p2["lon"], p2["lat"], 1],
        [p3["lon"], p3["lat"], 1]
    ])) < 0

while candidate_pillars:
    p_id = candidate_pillars.pop()
    triangle = connecting_triangles[p_id]
    p_index = np.where(triangle == p_id)[0][0]
    left = triangle[(p_index-1)%3]
    right = triangle[(p_index+1)%3]
    # Make sure we've labelled left & right correctly.
    if not clockwise_order(p_index, left, right, pillars):
        left, right = right, left
    # We wanna calculate the side lengths connected to 'p'.
    #       /| left
    #      / |
    #     /  |
    #    /   |
    # p ------ right
    # Use the law of sines! Given the internal angles of the triangle and
    # a single side length, the remaining side lengths can be calculated.
    update_distances(p_id, left, right, pillars, distance_map)
    final_coords[p_id] = calc_final_coords(
        left, p_id, pillars, final_coords, distance_map)
    visited_pillars.add(p_id)
    candidate_pillars |= fetch_new_candidates(
        triangles_by_pillar, distance_map, p_id, connecting_triangles)
    actual_dist = np.linalg.norm(
        get_pillar_coords(pillars, p_id)-get_pillar_coords(pillars, left))
    estimated_dist = get_dist(distance_map, p_id, left)
    ### Debugging to make sure we accurately reconstruct the triangle mesh when
    ### there's no error in the initial measurement.
    #if abs(actual_dist-estimated_dist)/actual_dist > 0.05:
    #    print("fuck,", len(visited_pillars))
    #    print(actual_dist, estimated_dist)
    #    print(pillars[p_id]["name"])
    #    print(pillars[left]["name"])
    #    print(pillars[right]["name"])
    #    break

Finally, re-draw the map based on the estimated distances.

In [24]:
"""These are the same as the `draw_pillars` and
`draw_mesh` functions defined above, except
coordinates are based on a `final_coords` dict."""
def draw_reconstructed_pillars(ctx, image, W, H, final_coords):
    pillar_paint = pixie.Paint(pixie.SOLID_PAINT)
    pillar_paint.color = pixie.Color(0, 0, 1, 1)
    ctx.fill_style = pillar_paint
    for coords in final_coords.values():
        draw_pillar(image, pillar_paint, W, H, coords)

def draw_reconstructed_mesh(ctx, W, H, final_coords, triangles):
    paint = pixie.Paint(pixie.SOLID_PAINT)
    paint.color = pixie.Color(1, 0, 0, 0.7)

    ctx.stroke_style = paint
    ctx.line_width = 0.5
    for triangle in triangles:
        can_draw = True
        for i in triangle:
            if i not in final_coords:
                can_draw = False
        if not can_draw:
            continue
        for (i, j) in [(0, 1), (1, 2), (2, 0)]:
            draw_line(ctx, W, H,
                      final_coords[triangle[i]],
                      final_coords[triangle[j]])

W, H = 300, 360
image = pixie.Image(W, H)
image.fill(pixie.Color(1, 1, 1, 1))
ctx = image.new_context()

draw_ireland(ctx, W, H, polys)

draw_reconstructed_pillars(ctx, image, W, H, final_coords)
draw_reconstructed_mesh(ctx, W, H, final_coords, triangles)
image.write_file("/tmp/reconstructed_map.png")

#debug_pillars = [pillar for i, pillar in enumerate(pillars)
#    if i in final_coords]
#draw_pillars(ctx, image, W, H, debug_pillars)
#draw_mesh(ctx, W, H, pillars, triangles)
#image.write_file("/tmp/goal_map.png")

In [137]:
print(len(visited_pillars))
print(len(pillars))

854
868


In [120]:
print(final_coords)

{763: array([0.616475  , 0.86062886]), 766: array([0.64250271, 0.87764365]), 764: array([0.62173289, 0.88401807]), 87: array([0.66509853, 0.83847079])}
